## Check Device

In [ ]:
!nvidia-smi

Wed Dec 21 01:48:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mount Google Drvie & OneDrive to Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!python -m pip install --upgrade pip setuptools wheel
# https://colab.research.google.com/github/jakiya99/Torrent-to-Onedrive/blob/main/new_torrents_download_to_onedrive.ipynb#scrollTo=Fil_bHuAD8S3
# https://github.com/jakiyaa/Torrent-to-Onedrive/blob/main/full%20tutorial.md
# https://github.com/jakiyaa/rclone-authenticate

!wget https://downloads.rclone.org/v1.60.1/rclone-v1.60.1-linux-amd64.deb
!apt install ./rclone-v1.60.1-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!mkdir onedrive
# 사용자 shell이 종료되어도 작동되기 위해 nohup(no hang up), & 사용
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

In [ ]:
!ls ./onedrive

## Install TTS package

In [ ]:
!pip install TTS==0.10.0

## Train HiFi-GAN

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.vocoder.configs import HifiganConfig
from TTS.tts.configs.shared_configs import BaseAudioConfig
from TTS.vocoder.datasets.preprocess import load_wav_data
from TTS.vocoder.models.gan import GAN

output_path = '/content/drive/MyDrive/glow-tts-dialect/model/hifigan/'

# define audio config
audio_config = BaseAudioConfig(
    sample_rate=22050,
    resample=True,
)

config = HifiganConfig(
    batch_size=64,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=5,
    epochs=1000,
    seq_len=8192,
    pad_short=2000,
    use_noise_augment=True,
    print_step=10,
    print_eval=False,
    mixed_precision=False,
    lr_gen=1e-4,
    lr_disc=1e-4,
    data_path="/content/onedrive/jss-dataset",
    output_path=output_path,
    audio=audio_config,
)

# init the audio processor
ap = AudioProcessor.init_from_config(config)

# load training samples
eval_samples, train_samples = load_wav_data(config.data_path, config.eval_split_size)

# init the model
model = GAN(config, ap)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

trainer.fit()